In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#https://www.linkedin.com/pulse/text-classification-using-bag-words-approach-nltk-scikit-rajendran

In [0]:
d=[]
while(1):
  d.append('1')

In [0]:
!pip install jupyter_http_over_ws
!jupyter serverextension enable --py jupyter_http_over_ws

In [0]:
!jupyter notebook \
  --NotebookApp.allow_origin='https://colab.research.google.com' \
  --port=8888 \
  --NotebookApp.port_retries=0

/usr/local/lib/python2.7/dist-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
Traceback (most recent call last):
  File "/usr/local/bin/jupyter-notebook", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/jupyter_core/application.py", line 267, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 657, in launch_instance
    app.initialize(argv)
  File "</usr/local/lib/python2.7/dist-packages/decorator.pyc:decorator-gen-7>", line 2, in initialize
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 87, in catch_config_error
    return method(app, *args, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/notebook/notebookapp.py", line 1368, in 

In [0]:
cd /content/drive/My\ Drive/Colab\ Notebooks/si650/hw4

/content/drive/My Drive/Colab Notebooks/si650/hw4


In [0]:
!ls

'Copy of hw4.ipynb'   sample_submission.csv   test.tsv
 hw4.ipynb	      submit.csv	      train.tsv
 new_test.csv	      temp.csv		      Untitled.ipynb


In [0]:
import time
import pandas as pd
import numpy as np
import scipy
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
#from autocorrect import Speller
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import re

In [0]:
np.random.seed(444)

In [0]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')
!pip install autocorrect
!pip install scipy

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
df = pd.read_csv('train.tsv', sep='\t')

In [0]:
df.head()

,label,text
0,0,"NASCAR organized the NASCAR Thunder 100 , a pa..."
1,1,"Like its sister project Wikipedia , Wiktionary..."
2,1,There are two ways to combine the many dispara...
3,1,"Rodolphus Agricola , Dutch scholar and humanis..."
4,0,Proxima Centauri was found to share the same p...


In [0]:
df1 = pd.read_csv('test.tsv', sep='\t', header=0)

In [0]:
%%time
i = 0
for index,content in df.iterrows():
  sms = content['text']
  sms = re.sub('[^A-Za-z]', ' ', sms)
  sms = sms.lower()
  tokenized_sms = word_tokenize(sms)
  # for word in tokenized_sms:
  #   if word in stopwords.words('english'):
  #       tokenized_sms.remove(word)
  stemmer = PorterStemmer()
  for i in range(len(tokenized_sms)):
    tokenized_sms[i] = stemmer.stem(tokenized_sms[i])
  sms_text = " ".join(tokenized_sms)
  df.loc[index,'new_text'] = sms_text
  i += 1
  if i % 10000 == 0:
    print(i)

CPU times: user 3h 51min 44s, sys: 2min 40s, total: 3h 54min 25s
Wall time: 3h 54min 37s


In [0]:
df.to_csv("temp.csv",index=False)

NameError: ignored

In [0]:
df = pd.read_csv("temp.csv")

In [0]:
# df.drop('Unnamed: 0',axis = 1,inplace = True)
df.head()

,label,text,new_text
0,0,"NASCAR organized the NASCAR Thunder 100 , a pa...",nascar organ nascar thunder pair exhibit lap r...
1,1,"Like its sister project Wikipedia , Wiktionary...",like sister project wikipedia wiktionari run t...
2,1,There are two ways to combine the many dispara...,are two way combin mani dispar protocol one wa...
3,1,"Rodolphus Agricola , Dutch scholar and humanis...",rodolphu agricola dutch scholar humanist lrb rrb
4,0,Proxima Centauri was found to share the same p...,proxima centauri found share same proper motio...


In [0]:
print(df.shape)
print(df.isnull().sum())

(416768, 3)
label         0
text          0
new_text    728
dtype: int64


In [0]:
df.drop('text',axis = 1,inplace=True)

In [0]:
for i in df['new_text']:
  try:
    if len(i) <1:
      print(i)
  except:
    print(i)

In [0]:
df.dropna(inplace=True)

In [0]:
df.shape

(416040, 2)

In [0]:
df1 = pd.read_csv('train.tsv', sep='\t')
# df1 = pd.read_csv("temp.csv")
# df1.dropna(inplace=True)
# df1.drop('text',axis = 1,inplace=True)

In [0]:
df_test = pd.read_csv('test.tsv', sep='\t')
# df_test = pd.read_csv("new_test.csv")
# df_test = pd.read_csv('test.tsv', sep='\t', header=0)

In [0]:
vectorizer = TfidfVectorizer(ngram_range=(1,4),lowercase=False,use_idf = False,max_df=0.6)
vectorizer.fit(df1['text'],df_test['text'])
X = vectorizer.transform(df1['text'])
X_test = vectorizer.transform(df_test['text'])

In [0]:
y = df1['label']

# split train and test data
from sklearn.model_selection import train_test_split
X_train, X_test1, y_train, y_test = train_test_split(X, y,test_size = 0.2)

In [0]:
from sklearn.model_selection import cross_validate
for i in range(1,10):
  SVM = svm.LinearSVC(C = i)
  cv_results = cross_validate(SVM, X, y, cv=3)
# predict the labels on validation dataset
  # predictions_SVM = clf.predict(X_test1)
# Use accuracy_score function to get the accuracy
  print(i, "SVM Accuracy Score -> ",cv_results[test_score])

In [0]:
SVM = svm.LinearSVC(C = 5)
SVM.fit(X,y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(X_test1)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_test)*100)
predictions_SVM = SVM.predict(X_test)
print(len(predictions_SVM))
print(sum(predictions_SVM == 1))
print(sum(predictions_SVM == 0))

SVM Accuracy Score ->  98.6479353120426
119092
63868
55224


In [0]:
submit = pd.DataFrame({"Id":df_test['id'],"Category":predictions_SVM})
submit.to_csv("submit.csv",index= False)

In [0]:
predictions_SVM = SVM.predict(X_test)
len(predictions_SVM)
print(sum(predictions_SVM == 1))
print(sum(predictions_SVM == 0))

65494
53598


In [0]:
for i in range(1,10):
  SVM = svm.LinearSVC(C = i)
  SVM.fit(X_train,y_train)
# predict the labels on validation dataset
  predictions_SVM = SVM.predict(X_test1)
# Use accuracy_score function to get the accuracy
  print(i," SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_test)*100)

1  SVM Accuracy Score ->  68.01113323895673
2  SVM Accuracy Score ->  67.76999304172566
3  SVM Accuracy Score ->  67.72680375266934
4  SVM Accuracy Score ->  67.6680183314538
5  SVM Accuracy Score ->  67.67401684382274
6  SVM Accuracy Score ->  67.69441178587711
7  SVM Accuracy Score ->  67.69201238092954
8  SVM Accuracy Score ->  67.67521654629653


In [0]:
predictions_SVM = SVM.predict(X_test)
# Use accuracy_score function to get the accuracy
# print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_test)*100)

In [0]:
SVM

LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=0, tol=1e-06,
          verbose=0)

In [0]:
temp = pd.read_csv('submit.csv')

119092

In [0]:
predictions_SVM[0:20]

array([1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0])

In [0]:
num_pos = (df['label']==1).sum()
num_neg = (df['label']==0).sum()
gb = XGBClassifier(n_estimators = 700, scale_pos_weight = num_neg/num_pos, max_depth = 5)#GradientBoostingClassifier(n_estimators=700, max_depth=5)
gb.fit(X_train,y_train)
prediction_xgb = gb.predict(X_test)
print("SVM Accuracy Score -> ",accuracy_score(prediction_xgb, y_test)*100)

In [0]:
model = LogisticRegression()
model.fit(X_train,y_train)
predict_log = model.predict(X_test)
print("SVM Accuracy Score -> ",accuracy_score(predict_log, y_test)*100)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


SVM Accuracy Score ->  67.88610069544595
